In [3]:
import pandas as pd
import nltk
import re
from collections import defaultdict, Counter
import random
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [4]:
df=pd.read_csv("/Users/meenakshiganesan/Karthik/KarthikProgramming/Cp_repo/Swami Saranam/Codeforces/bny/Eminem.csv")
lyrics_text = " ".join(df['Lyric'].dropna().astype(str))

In [5]:
lyrics_text = " ".join(df['Lyric'].dropna().astype(str))

In [6]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text


In [7]:
cleaned_text = clean_text(lyrics_text)
tokens = nltk.word_tokenize(cleaned_text)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/meenakshiganesan/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
tokens = nltk.word_tokenize(cleaned_text)
print("Sample tokens:", tokens[:20])

Sample tokens: ['look', 'i', 'was', 'gon', 'na', 'go', 'easy', 'on', 'you', 'not', 'to', 'hurt', 'your', 'feelings', 'but', 'im', 'only', 'going', 'to', 'get']


In [ ]:
def build_ngram_model(tokens, n=3):
    model = defaultdict(Counter)
    for i in range(len(tokens)-n):
        prefix = tuple(tokens[i:i+n-1])
        next_word = tokens[i+n-1]
        model[prefix][next_word] += 1
    return model

In [ ]:
trigram_model = build_ngram_model(tokens, n=3)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
model = AutoModelForCausalLM.from_pretrained("distilgpt2")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

2025-09-22 14:34:21.040526: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758551661.306871      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758551661.382824      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
def generate_hybrid_lyrics(trigram_model, num_lines=5, line_length=8):
    lyrics = []
    for _ in range(num_lines):
        seed = random.choice(list(trigram_model.keys()))
        output_words = list(seed)
        
        for _ in range(line_length):
            prefix = tuple(output_words[-2:])
            if prefix in trigram_model:
                next_word = random.choices(
                    list(trigram_model[prefix].keys()),
                    weights=trigram_model[prefix].values()
                )[0]
                output_words.append(next_word)
            else:
                break
        line_text = " ".join(output_words)
        input_ids = tokenizer(line_text, return_tensors="pt").input_ids
        output = model.generate(
            input_ids,
            max_length=len(input_ids[0])+10,
            do_sample=True,
            temperature=0.8,
            top_k=50,
            top_p=0.95
        )
        polished_line = tokenizer.decode(output[0], skip_special_tokens=True)
        lyrics.append(polished_line)
        
    return "\n".join(lyrics)

In [ ]:
generated = generate_hybrid_lyrics(trigram_model, num_lines=5, line_length=8)
print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

na name yourself after a damn gun and an emergency room. I want to be a person that I
my cousin youre dead inside and you got it youd get to have a better day.
gunman opened fire newlyreleased surveillance video shows the moments when he was shot while taking the fatal shot.

ayo i sit in the strip club dont forget about that!
mummy costume on stage screamin how people hate me cause she didn't like me.
